In [8]:
from collections import Counter
import pandas as pd
import numpy as np
import time
import datetime

In [9]:
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251')

In [10]:
cases = df['CaseID'].unique()

## P2P - 1 Поставщики с высоким уровнем возвратов

In [11]:
def weighted_mean(df):
    k = 0
    cases = df['CaseID'].unique()
    ret_cases = []

    d_n = {}
    d_ret = {}
    for case in cases:
        c = df.iloc[np.where(df['CaseID'] == case)]
        act = c['Activity Category'].tolist()
        sup = c['Supplier'].tolist()[0]
        if sup in d_n:
            d_n[sup] += 1
        else:
            d_n[sup] = 1
        if 'Поступление материала-Возврат' in act:
            if sup in d_ret:
                d_ret[sup] += 1
            else:
                d_ret[sup] = 1
            ret_cases.append(c['Supplier'].tolist()[0])
        
    for key in d_ret.keys():
        d_ret[key] /= (1212 - d_n[key])
    
    return sorted(d_ret.items(), key=lambda x: x[1])
weighted = weighted_mean(df)

In [12]:
# P2P - 1 Поставщики с высоким уровнем возвратов
check = True
danger_suppliers = []
quantity_of_danger_sup = 0
for i in range(len(weighted)): # проверяем сколько постващиков имеют высокий уровень возвратов
    if(check):
        main_weight = weighted[i][1]/2
        check = False
    if(weighted[i][1] > main_weight):
        quantity_of_danger_sup += 1
        danger_suppliers.append(weighted[i][0])
    else:
        break
print("Количество поставщиков с высоким уровнем возвратов:", quantity_of_danger_sup)

Количество поставщиков с высоким уровнем возвратов: 98


## P2P - 2 Процент заказов у поставщиков с высоким уровнем возвратов

In [13]:
# P2P - 2 Процент заказов у поставщиков с высоким уровнем возвратов
n = 0
quantity_of_danger_orders = 0
for i in range(len(df["CaseID"])):
    if(cases[n] == df["CaseID"][i]):
        if(df['Supplier'][i] in danger_suppliers):
            quantity_of_danger_orders += 1
        i = 0
        n += 1
    if(n == len(cases)):
        break
print("Количество заказов у поставщиков с высоким уровнем возвратов:", quantity_of_danger_orders)
print("Процент заказов у поставщиков с высоким уровнем возвратов:", round(quantity_of_danger_orders/len(cases) * 100), "%")

Количество заказов у поставщиков с высоким уровнем возвратов: 2883
Процент заказов у поставщиков с высоким уровнем возвратов: 36 %


## P2P - 3 Среднее время подтверждения заказа

In [14]:
# P2P - 3 среднее время подтверждения заказа
cases = df['CaseID'].unique()
cases = list(cases)
case_time_list = []
for i in range(len(cases)):
    case_time_list.append([cases[i], False]) # Создаём вспомогательный массив
approved_times = []
approved = ['Заказ на поставку согласован 1', 'Заказ на поставку согласован 2']
df_time_sort = df.sort_values(by = 'Event end')
df_time_sort = df_time_sort.reset_index(drop = True) # Сортриуем таблицу
for j in range(len(df_time_sort) - 1, -1, -1): # итерируемся с конца табоицы
    case_index = cases.index(df_time_sort['CaseID'][j])
    if(df_time_sort["Activity"][j] in approved and not case_time_list[case_index][1]): # проверяем есть ли согласование заказа
        try: # считаем время согласования
            approved_time = time.mktime(time.strptime(str(df_time_sort['Event end'][j]), "%Y-%m-%d %H:%M:%S.%f"))
        except ValueError:
            approved_time = time.mktime(time.strptime(str(df_time_sort['Event end'][j]), "%Y-%m-%d %H:%M:%S"))
        case_time_list[case_index].append(approved_time)
        case_time_list[case_index][1] = True
    if(df_time_sort["Activity"][j] == 'Заказ на поставку создан'): # проверяем время создания заказа и считаем его
        try:
            starting_time = time.mktime(time.strptime(str(df_time_sort['Event end'][j]), "%Y-%m-%d %H:%M:%S.%f"))
        except ValueError:
            starting_time = time.mktime(time.strptime(str(df_time_sort['Event end'][j]), "%Y-%m-%d %H:%M:%S"))
        case_time_list[case_index].append(starting_time)
        
cases_without_confirm = 0
for n in range(len(case_time_list)): # отнимаем время согласования и создания заявки у всех кейсов
    if(len(case_time_list[n]) == 4):
        approved_times.append(case_time_list[n][2] - case_time_list[n][3])
    if(len(case_time_list[n]) == 3):
        cases_without_confirm += 1

if(len(approved_times) == 0):
    print("Нету согласованных заказов")
else:
    print("среднее время подтверждения заказа:", sum(approved_times)/len(approved_times), "сек.")
    print("Процент кейсов без согласования:", 
          round(cases_without_confirm/(cases_without_confirm + len(approved_times)) * 100), "%")
    print("Процент согласованных кейсов:", 
          round(len(approved_times)/(cases_without_confirm + len(approved_times)) * 100), "%")

среднее время подтверждения заказа: 1021990.7892390012 сек.
Процент кейсов без согласования: 58 %
Процент согласованных кейсов: 42 %


## P2P - 4 Какой процент заказов — это услуги, а какой закупка товаров

In [15]:
# P2P - 4 Какой процент заказов — это услуги, а какой закупка товаров
cases = df['CaseID'].unique()
n = 0
services = 0
goods = 0
services_total_amount = 0
goods_total_amount = 0
Purchase_type = df['Purchase type'].unique()
for i in range(len(df["CaseID"])):
    if(cases[n] == df["CaseID"][i]):
        if(df['Purchase type'][i] == Purchase_type[0]):
            services += 1
            if(not np.isnan(df["Amount"][i])):
                services_total_amount += df["Amount"][i]
        else:
            if(not np.isnan(df["Amount"][i])):
                goods_total_amount += df["Amount"][i]
            goods += 1
        i = 0
        n += 1
    if(n == len(cases)):
        break
serv_per = services/(services + goods) * 100
goods_per = goods/(services + goods) * 100
print("процент заказов, которые являются услугами:", round(serv_per), "%",
      "\nпроцент заказов, которые являются закупкой товаров:", round(goods_per), "%")

процент заказов, которые являются услугами: 48 % 
процент заказов, которые являются закупкой товаров: 52 %


## P2P - 5 Процент товаров, которые поставляются исключительно внутренними поставщиками

In [16]:
# P2P - 5 процент товаров которые поставляются исключительно внутренними поставщиками
Material = df['Material'].unique() # массив всех материалов
n = 0
total_amount_domestic_mat = 0
for i in range(len(df["CaseID"])):
    if(Material[n] == df["Material"][i]):
        if(df["Supplier_type"][i] == 'Внутренние'): # проверяем является ли поставщик внутренним
            total_amount_domestic_mat += 1
            i = 0
            n += 1
        else:
            i = 0
            n += 1
    if(n == len(Material)):
        break
    #print("i = ", i)
print("общее количество внутренних товаров:", total_amount_domestic_mat)
print("процент товаров поставляющихся внутренними поставщиками по отношению ко всем товарам:", 
      round(total_amount_domestic_mat/len(Material) * 100), "%")

общее количество внутренних товаров: 41
процент товаров поставляющихся внутренними поставщиками по отношению ко всем товарам: 2 %


## P2P - 6 Общее количество потраченных денег

In [17]:
# P2P - 6 Общее количество потраченных денег
n = 0
total_amount = 0
for i in range(len(df["CaseID"])):
    if(cases[n] == df["CaseID"][i]):
        if(not np.isnan(df["Amount"][i])):
            total_amount += df["Amount"][i]
        i = 0
        n += 1
    if(n == len(cases)):
        break
print("общее количество потраченных денег:", total_amount)

общее количество потраченных денег: 2775166966.7699986


## P2P - 7 Среднее количество изменений заказа

In [18]:
# P2P - 7 Среднее количество изменений заказа
changes = ['Заказ на поставку бессрочно заблокирован', 'Заказ на поставку изменен: лимит на недопоставку', 
             'Заказ на поставку изменен: группа закупки', 'Заказ на поставку изменен: лимит на сверх-поставку',
             'Заказ на поставку изменен: материал', 'Заказ на поставку изменен: налоговые условия',
             'Заказ на поставку изменен: запланированный срок доставки в днях', 
              'Заказ на поставку изменен: эффективная стоимость', 'Заказ на поставку изменен: уменьшена стоимость',
             'Заказ на поставку изменен: уменьшена цена', 'Заказ на поставку изменен: уменьшено количество',
             'Заказ на поставку изменен: увеличена стоимость', 'Заказ на поставку изменен: увеличено количество',
             'Заказ на поставку изменен: увеличена цена', 'Заказ на поставку изменен: срока предоставления скидки',
             'Заказ на поставку изменен: условия оплаты', 'Заказ на поставку изменен: завод', 
              'Заказ на поставку изменен: статус наличия счета', 'Заказ на поставку изменен: валюта',
             'Заказ на поставку изменен: поставщик', 'Заказ на поставку изменен: тип документа']

quantity_of_changes = 0
for i in range(len(df["CaseID"])):
    if(df["Activity"][i] in changes):
        quantity_of_changes += 1
print("всего изменений заказа:", quantity_of_changes)
print("среднее количество изменений заказа:", quantity_of_changes/len(cases))

всего изменений заказа: 2255
среднее количество изменений заказа: 0.2809968847352025


## P2P - 8 Количество неправильно завершенных частичных поставок

In [19]:
# P2P - 8 Количество неправильно завершенных частичных поставок
activ = ['Поступление материала-Возврат-Частичная поставка', 'Поступление материала-Получение-Завершающая поставка']
quantity_of_wrong_cases = 0
for i in range(len(df["CaseID"])):
    bool = True
    curr_case = df["CaseID"][i]
    if(df["Activity"][i] == activ[0]):
        j = i
        while(curr_case == df["CaseID"][j]):
            if(df["Activity"][i] == activ[1]):
                bool = False
                break
            j += 1
        if(bool):
            quantity_of_wrong_cases += 1
print("количество неправильных кейсов:", quantity_of_wrong_cases)

количество неправильных кейсов: 610


## P2P - 9 Количество денег, потраченных на  заказы/услуги в процентом соотношении

In [20]:
# P2P - 9 Количество денег потраченных на  заказы/услуги в процентом соотношении
print("количество денег, которые были потрачены на услуги в процентом соотношении:", 
      int(round(services_total_amount/total_amount * 100)), "%",
     "\nколичество денег, которые были потрачены на товары в процентом соотношении:",
     int(round(goods_total_amount/total_amount * 100)), "%")

количество денег, которые были потрачены на услуги в процентом соотношении: 34 % 
количество денег, которые были потрачены на товары в процентом соотношении: 66 %


## P2P-10 Матожидание возврата

In [21]:
# P2P-10 Матожидание возврата
# first element of suppliers_list is a name of supplier, second is sum of all returns, third is a quantity of returns
# and fourth element is quantity of orders
suppliers = df['Supplier'].unique()
suppliers = list(suppliers)
suppliers_list = []
cases_summed = []
for i in range(len(suppliers)):
    suppliers_list.append([suppliers[i], 0, 0, 0])
for j in range(len(df["CaseID"])):
    supplier_index = suppliers.index(df['Supplier'][j])
    if(df['CaseID'][j] not in suppliers_list[supplier_index]):
        suppliers_list[supplier_index].append(df['CaseID'][j])
        suppliers_list[supplier_index][3] += 1
    if(df['Activity Category'][j] == 'Поступление материала-Возврат'):
        if(df['CaseID'][j] not in cases_summed):
            if(not np.isnan(df["Amount"][j])):
                suppliers_list[supplier_index][1] += df["Amount"][j]    
            cases_summed.append(df['CaseID'][j])
            suppliers_list[supplier_index][2] += 1
expec = 0
for n in range(len(suppliers_list)):
    expec += suppliers_list[n][2]/suppliers_list[n][3] * suppliers_list[n][1]
print("Матожидание возврата:", expec)

Матожидание возврата: 82585037.34873027
